In [11]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
import re
from datetime import datetime
from decimal import Decimal, InvalidOperation
import base64
from datetime import datetime, timedelta
import pytz


pd.options.display.float_format = '{:,.14f}'.format

In [12]:
load_dotenv()
TOKEN_URL = 'https://api.ticket360.com.br/auth/oauth/access_token'
API_URL = 'https://api.ticket360.com.br'
EVENT_ID = '30617'
EVENT_NAME = 'Camarote-essepe-2026-grupo-especial-thiaguinho'
MAX_RETRIES = 3
TIMEOUT = 30
DATA_DIR = "ticket_data"
CONSOLIDATED_FILE = os.path.join(DATA_DIR, "consolidated.json")

In [13]:
def get_access_token():
    """Obtém token de acesso com tratamento de erros"""
    for attempt in range(MAX_RETRIES):
        try:
            auth_string = f"{os.getenv('CLIENT_ID')}:{os.getenv('CLIENT_SECRET')}"
            auth_base64 = base64.b64encode(auth_string.encode()).decode()
            
            response = requests.post(
                TOKEN_URL,
                headers={
                    "Authorization": f"Basic {auth_base64}",
                    "Content-Type": "application/x-www-form-urlencoded"
                },
                data={"grant_type": "client_credentials"},
                timeout=TIMEOUT
            )
            response.raise_for_status()
            return response.json().get("access_token")
        except requests.exceptions.Timeout:
            print(f"Timeout (tentativa {attempt + 1}/{MAX_RETRIES})")
            if attempt < MAX_RETRIES - 1:
                time.sleep(5)
        except Exception as e:
            print(f"Erro ao obter token: {type(e).__name__} - {str(e)}")
            return None
    return None

In [14]:
def fetch_report(token, start_date=None, end_date=None):
    """Busca relatório da API com paginação e aplica filtro local de datas"""
    try:
        base_url = f"{API_URL}/sales/reports/consolidated/{EVENT_ID}?filter=status=paid&ticket.status=active"
        
        offset = 0
        limit = 1000
        all_sales = []
        
        while True:
            url = f"{base_url}&limit={limit}&offset={offset}"
            response = requests.get(
                url,
                headers={"Authorization": f"Bearer {token}"},
                timeout=TIMEOUT
            )
            response.raise_for_status()
            data = response.json()
            sales = data.get('sales', [])
            all_sales.extend(sales)
            
            if len(sales) < limit:
                break
            offset += limit
        
        # Converte para DataFrame para tratamento de datas
        df_sales = pd.DataFrame(all_sales)
        if "date" in df_sales.columns and not df_sales.empty:
            # Se a coluna for numérica -> timestamp em ms
            if pd.api.types.is_numeric_dtype(df_sales["date"]):
                df_sales["date"] = pd.to_datetime(df_sales["date"], errors="coerce", unit="ms")
            else:
                # Se já vier como string ISO
                df_sales["date"] = pd.to_datetime(df_sales["date"], errors="coerce")
            
            # Se existirem filtros de start_date / end_date, aplicar
            if start_date:
                df_sales = df_sales[df_sales["date"].dt.date >= pd.to_datetime(start_date).date()]
            if end_date:
                df_sales = df_sales[df_sales["date"].dt.date <= pd.to_datetime(end_date).date()]
        
        return {"sales": df_sales.to_dict(orient="records")}
    
    except Exception as e:
        print(f"Erro ao buscar relatório: {type(e).__name__} - {str(e)}")
        return None


In [15]:
def normalize_dates(df: pd.DataFrame, date_column: str = "date") -> pd.DataFrame:
    """Normaliza datas para o formato ISO-8601 (YYYY-MM-DDTHH:MM:SS±HH:MM)."""
    if date_column not in df.columns:
        return df
    
    # Converte para datetime, forçando UTC quando possível
    df[date_column] = pd.to_datetime(df[date_column], utc=True, errors="coerce")
    
    # Converte para string no formato ISO
    df[date_column] = df[date_column].dt.strftime("%Y-%m-%dT%H:%M:%S%z")
    
    # Ajusta fuso horário para ter o ":" no offset (+0000 -> +00:00)
    df[date_column] = df[date_column].str.replace(
        r"(\+)(\d{2})(\d{2})$", r"\1\2:\3", regex=True
    )
    
    return df

In [16]:
def remove_today_data(df):
    hoje = datetime.now().date()

    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce", utc=True)
        df["date_only"] = df["date"].dt.date
        df = df[df["date_only"] != hoje].drop(columns=["date_only"])
    else:
        print("'Date' column not found")

    return df 

In [17]:
def load_consolidated_ndjson(filepath):
    # Lê dados de um arquivo JSON no formato array
    if not os.path.exists(filepath):
        return []

    try:
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
            if isinstance(data, list):
                return data
            elif isinstance(data, dict):
                return [data]  # caso o arquivo tenha apenas um registro
            else:
                print(f"Formato inesperado no arquivo {filepath}")
                return []
    except json.JSONDecodeError as e:
        print(f"Erro ao carregar {filepath}: {e}")
        return []



In [18]:
def append_to_consolidated_json(filepath, new_records):
    #Add new records to consolidated.json file keeping the array format
    existing_data = load_consolidated_ndjson(filepath)
    updated_data = existing_data + new_records

    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(updated_data, f, indent=2, ensure_ascii=False, default=str)

In [19]:
def get_last_date_from_consolidated():
    records = load_consolidated_ndjson(CONSOLIDATED_FILE)
    if not records:
        return None
    df = pd.DataFrame(records)
    if "date" not in df.columns or df.empty:
        return None
    df["date"] = pd.to_datetime(df["date"], errors="coerce", utc=True)
    return df["date"].max().date()


In [20]:
def save_to_json(data, filename):
    """Salva dados em um arquivo JSON com tratamento de datas"""
    def convert_timestamps(obj):
        if isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        if isinstance(obj, datetime):
            return obj.isoformat()
        return obj
    
    with open(filename, 'w') as f:
        if isinstance(data, pd.DataFrame):
            json_data = data.to_dict(orient='records')
            json.dump(json_data, f, indent=2, default=convert_timestamps)
        else:
            json.dump(data, f, indent=2, default=convert_timestamps)

In [21]:
def consolidate_data(new_data_file):
    """Consolida novos dados com o arquivo principal"""
    # Carregar dados existentes
    df_new = pd.read_json(CONSOLIDATED_FILE)

    if os.path.exists(CONSOLIDATED_FILE) and os.path.getsize(CONSOLIDATED_FILE) > 2:
        try:
            df_old = pd.read_json(CONSOLIDATED_FILE)
        except ValueError:
            # Se estiver corrompido, tenta reparar removendo qualquer “lixo” após o colchete
            with open(CONSOLIDATED_FILE, "r", encoding="utf-8") as f:
                raw = f.read()
            if "]" in raw:
                raw = raw[:raw.rfind("]")+1]
                df_old = pd.read_json(pd.io.common.StringIO(raw))
            else:
                df_old = pd.DataFrame()
    else:
        df_old = pd.DataFrame()

    # Concatena
    if not df_old.empty:
        df_all = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_all = df_new.copy()

    # Remoção de duplicatas (ajuste as chaves conforme sua noção de unicidade)
    keys_prioridade = ['ticket.id', 'ticket.code', 'id']
    subset = [c for c in keys_prioridade if c in df_all.columns]
    if subset:
        # Mantém o mais recente pela coluna 'date' quando houver
        if 'date' in df_all.columns:
            df_all = df_all.sort_values('date')
        df_all = df_all.drop_duplicates(subset=subset, keep='last')

    # NÃO TOCA no tipo de 'date' (se já está em ms, mantém; se já é string, mantém)
    # Salva como JSON array bonito
    df_all.to_json(CONSOLIDATED_FILE, orient='records', force_ascii=False, indent=2)

    return df_all

In [22]:
def get_latest_date():
    """Obtém a última data do dataset consolidado"""
    if not os.path.exists(CONSOLIDATED_FILE):
        return None
    
    try:
        df = pd.read_json(CONSOLIDATED_FILE)
        if 'date' not in df.columns or df.empty:
            return None
        if pd.api.types.is_integer_dtype(df['date']):
            s = pd.to_datetime(df['date'], unit='ms', utc=True)
        else:
            s = pd.to_datetime(df['date'],errors='coerce',utc=True)

        if s.isna().all():
            return None 
        
        return s.max().date()
    except Exception as e:
        print(f"Erroe to get last date: {e}")
        return None

        


In [23]:
def save_incremental_file(data, prefix="incremental"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join(DATA_DIR, f"{prefix}_{timestamp}.json")
    save_to_json(data, filename)
    print(f"Incremental data saved to {filename}")

In [24]:
def run_incremental_pipeline():
    token = get_access_token()
    if not token:
        print("Error to get token")
        return 
    
    # Requisição à API
    report_data = fetch_report(token)
    if not report_data or "sales" not in report_data:
        print("No data returned from API")
        return
    
    # Converter para DataFrame e normalizar
    df = pd.DataFrame(report_data["sales"])
    if df.empty:
        print("No sales data found")
        return
    
    df = normalize_dates(df, date_column="date")

    # Variáveis principais
    last_date = get_last_date_from_consolidated()   # data mais recente já armazenada
    if last_date is None:
        print("No consolidated data found")
        return
    
    start_date = last_date + timedelta(days=1)      # coleta deve iniciar a partir do próximo dia
    today = datetime.now().date()

    # Filtrar registros somente após a last_date
    df_filtered = df[df["date"].apply(lambda x: pd.to_datetime(x).date() >= start_date)]

    # Remover registros de hoje
    df_filtered = remove_today_data(df_filtered)

    if df_filtered.empty:
        print("Already up to date (no new data)")
        return
    
    # Data mais antiga disponível no df filtrado
    min_date = df_filtered["date"].apply(lambda x: pd.to_datetime(x).date()).min()

    if min_date == today:
        print("Already up to date (only today’s data available)")
        return
    
    if min_date > last_date and min_date != today:
        # Append ao consolidated.json
        append_to_consolidated_json(CONSOLIDATED_FILE, df_filtered.to_dict(orient="records"))
        # Salvar incremental separado
        save_incremental_file(df_filtered)
        print(f"Incremental file created: {CONSOLIDATED_FILE} ({len(df_filtered)} records)")
    else:
        print("Already up to date (no valid new data)")


In [30]:
if __name__ == "__main__":
    run_incremental_pipeline()

Already up to date (no new data)


C:\Users\victo\AppData\Local\Temp\ipykernel_16824\3889756053.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = pd.to_datetime(df["date"], errors="coerce", utc=True)
C:\Users\victo\AppData\Local\Temp\ipykernel_16824\3889756053.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date_only"] = df["date"].dt.date


In [ ]:
'''
def delete_records_by_date(file_path, target_date):
    """
    Deleta registros onde 'date' é igual à data especificada
    target_date: string no formato 'YYYY-MM-DD' (ex: '2025-08-31')
    """
    try:
        # Ler o arquivo JSON como array
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print(f"Total de registros antes: {len(data)}")
        
        # Filtrar registros que NÃO têm a data alvo
        filtered_data = []
        deleted_count = 0
        
        for record in data:
            record_date = record.get('date', '')
            
            # Converter para string se for timestamp
            if isinstance(record_date, (int, float)):
                try:
                    dt = datetime.fromtimestamp(record_date / 1000)
                    record_date_str = dt.strftime('%Y-%m-%d')
                except:
                    record_date_str = ''
            else:
                record_date_str = str(record_date)
            
            # Manter apenas registros que NÃO têm a data alvo
            if target_date not in record_date_str:
                filtered_data.append(record)
            else:
                deleted_count += 1
        
        # Salvar o array filtrado de volta no arquivo
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(filtered_data, f, indent=2, ensure_ascii=False)
        
        print(f"Registros deletados: {deleted_count}")
        print(f"Total de registros depois: {len(filtered_data)}")
        print(f"Arquivo atualizado: {file_path}")
        
        return deleted_count
        
    except Exception as e:
        print(f"Erro: {e}")
        return 0

# Exemplo de uso:
delete_records_by_date("ticket_data/consolidated.json", "2025-09-02")

'''

Total de registros antes: 782
Registros deletados: 2
Total de registros depois: 780
Arquivo atualizado: ticket_data/consolidated.json


2